## 鄱阳湖水体区域识别
通过计算归一化水体指数 *NDWI* 指数提取鄱阳湖水体区域。

### 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

### *Landsat-8* 数据检索
使用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=2fd161" target="_blank">aie.Image</a>* 引用单景 *Landsat-8* 数据，并对数据进行去云处理。

In [2]:
def removeLandsatCloud(image):
    cloudShadowBitMask = (1 << 4)
    cloudsBitMask = (1 << 3)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(aie.Image(cloudShadowBitMask)).eq(aie.Image(0)).And(qa.bitwiseAnd(aie.Image(cloudsBitMask)).eq(aie.Image(0)))
    return image.updateMask(mask)

In [3]:
image = aie.Image('LC08_L2SP_121040_20210605_20210614_02_T1')
img = removeLandsatCloud(image)

### 水体提取
通过 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=281f4e" target="_blank">aie.Image.normalizedDifference</a>* 函数实现归一化水体指数（ *NDWI* ）的计算。利用  *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=9015df" target="_blank">aie.Image.where</a>* 、 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=9c1610" target="_blank">aie.Image.lte</a>* 、 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=c0fc05" target="_blank">aie.Image.gt</a>* 实现水体与非水体的二分类提取。

In [4]:
ndwi=img.normalizedDifference(['SR_B3', 'SR_B5'])
water=ndwi.where(ndwi.lte(aie.Image(0.0)),aie.Image(0)).where(ndwi.gt(aie.Image(0.0)),aie.Image(1))

In [5]:
map = aie.Map(
    center=image.getCenter(),
    height=800,
    zoom=7
)

# 水体区域为蓝色
vis_params = {
    'min': 0,
    'max': 1,
    'palette' : [
        '#a1a1a1', '#0000ff'
    ]
}

map.addLayer(
    water,
    vis_params,
    'water',
    bounds=image.getBounds()
)
map

Map(center=[28.861606865294476, 116.52096047179978], controls=(ZoomControl(options=['position', 'zoom_in_text'…